In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, classification_report

In [3]:
answer_data_path = '/content/drive/MyDrive/Research/running_posture/answer_annotated_data.csv'
wrong_data_path = '/content/drive/MyDrive/Research/running_posture/wrong_annotated_data.csv'

In [4]:
answer_data = pd.read_csv(answer_data_path)
wrong_data = pd.read_csv(wrong_data_path)

In [5]:
data = pd.concat([answer_data, wrong_data], ignore_index=True)

In [6]:
data.set_index('Time/Frame', inplace=True)

In [7]:
# 'Label' 칼럼 제외 정규화
features = data.columns.difference(['Label'])
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

In [8]:
# Label이랑 scaling 한 데이터 결합

data_scaled = pd.DataFrame(scaled_data, columns=features, index=data.index)
data_scaled['Label'] = data['Label'].values

In [9]:
# 데이터 LSTM 모델 입력 형식 변환
def create_dataset_lstm(dataset, labels, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step):
        dataX.append(dataset[i:(i + time_step)])
        dataY.append(labels[i + time_step])
    return np.array(dataX), np.array(dataY)

In [10]:
time_step = 10
X_lstm, y_lstm = create_dataset_lstm(data_scaled[features].values, data_scaled['Label'].values, time_step)

In [11]:
X_train_lstm, X_temp_lstm, y_train_lstm, y_temp_lstm = train_test_split(X_lstm, y_lstm, test_size=0.4, random_state=42, stratify=y_lstm)
X_val_lstm, X_test_lstm, y_val_lstm, y_test_lstm = train_test_split(X_temp_lstm, y_temp_lstm, test_size=0.5, random_state=42, stratify=y_temp_lstm)

In [12]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(time_step, X_lstm.shape[2])))
model_lstm.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
history = model_lstm.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=32, validation_data=(X_val_lstm, y_val_lstm))

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.5641 - loss: 0.6977 - val_accuracy: 0.7383 - val_loss: 0.6570
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7303 - loss: 0.6447 - val_accuracy: 0.8785 - val_loss: 0.6052
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8994 - loss: 0.5761 - val_accuracy: 0.9252 - val_loss: 0.4883
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9104 - loss: 0.4426 - val_accuracy: 0.9252 - val_loss: 0.2845
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9136 - loss: 0.2690 - val_accuracy: 0.9252 - val_loss: 0.2084
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9436 - loss: 0.1481 - val_accuracy: 0.9346 - val_loss: 0.1743
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9341 - loss: 0.1501 - val_accuracy: 0.9813 - val_loss: 0.1231
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9702 - loss: 0.0979 - val_accuracy: 0.9533 - va

In [15]:
# vali 셋 평가

y_val_pred_lstm = (model_lstm.predict(X_val_lstm) > 0.5).astype("int32")
accuracy_val_lstm = accuracy_score(y_val_lstm, y_val_pred_lstm)
report_val_lstm = classification_report(y_val_lstm, y_val_pred_lstm)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step


In [16]:
# test 셋 평가

y_test_pred_lstm = (model_lstm.predict(X_test_lstm) > 0.5).astype("int32")
accuracy_test_lstm = accuracy_score(y_test_lstm, y_test_pred_lstm)
report_test_lstm = classification_report(y_test_lstm, y_test_pred_lstm)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


In [17]:
print(f'Validation Accuracy: {accuracy_val_lstm}')
print(f'Validation Classification Report:\n{report_val_lstm}')
print('\n')
print(f'Test Accuracy: {accuracy_test_lstm}')
print(f'Test Classification Report:\n{report_test_lstm}')

Validation Accuracy: 0.9813084112149533
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        55
           1       0.96      1.00      0.98        52

    accuracy                           0.98       107
   macro avg       0.98      0.98      0.98       107
weighted avg       0.98      0.98      0.98       107



Test Accuracy: 0.9906542056074766
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        55
           1       0.98      1.00      0.99        52

    accuracy                           0.99       107
   macro avg       0.99      0.99      0.99       107
weighted avg       0.99      0.99      0.99       107

